In [1]:
#import matplotlib as matplot
#import seaborn
#import bokeh
#import keras
import numpy
#import scipy
import pandas as pd
from get_tree_data import tree_data as tree_import
from get_tree_data import sort_DXDM_cols
import re
# import block

In [2]:
class dataTree : pass

In [3]:
tree_root = dataTree()
tree = tree_import.tree

In [81]:
ncs1 = pd.read_csv("ICPSR_06693-V6/ICPSR_06693/DS0001/06693-0001-Data.tsv", sep = "\t")
# ncs1 includes all of the symptom data from the NCS Data
ncs1 = pd.DataFrame(ncs1)
#turn into pandas DF
ncsDXDM = pd.read_csv("ICPSR_06693-V6/ICPSR_06693/DS0002/06693-0002-Data.tsv", sep = "\t")
# ncs1 includes all of the symptom data from the NCS Data
ncsDXDM = pd.DataFrame(ncsDXDM)
#turn into pandas DF
ncsDXDM = ncsDXDM.reindex(sort_DXDM_cols(ncsDXDM.columns), axis=1)

# reindex according to codebook

CASEID
SEASON[SWIN, SSPR, SSUM, SFAL]
AGO/AGR
ALC
AAB/ASP
BP1
CDLT
DEP
DRG
DYS
GAD
MAN
PD/PT
SIM/SOC
PTSD
NAP
AG/OLDAG
ED
EMP/SELFEMP/WKHRS
HH
IN/PV
MAR
RACE/RWH/RBL/HRSP/ROTHR
R4
REL
SEX
STR/SECU/P1FWT/P2WTV2/P2WTV3/OLDP2FWt

In [270]:
def get_tree_obj_array(var, root, tree, dxdm):
    start = 10000000
    end = -10000000
      # setting start & end to -10000000 so they throw errors if not changed
    tmpDF = pd.DataFrame(columns = ['VarName', 'Description', 'Root_DF', 'Start', 'End', 'DataFrame', "recursion_flag"])
    for x in range (3, len(tree)):
        if len(tree[x]) > 3: 
            # If List has sublists
            var2 = re.sub('[^ A-Za-z0-9]+', '', tree[x][0])
            var2 = re.sub('[ ]+', '_', var2)
              #Make sublist variable name
            start_end = get_tree_obj_array(var2, root, tree[x], dxdm)
            if x == 3: 
                start = start_end[0]
                # first iteration gives start value
            if x+1 == len(tree):
                end = start_end[1]
                # last itereation gives end value
            if dxdm: 
                tmpDF.loc[len(tmpDF)] = [var2, tree[x][0], var, start_end[0], start_end[1], 'ncsDXDM', 1]
            else:
                tmpDF.loc[len(tmpDF)] = [var2, tree[x][0], var, start_end[0], start_end[1], 'ncs1', 1]
            # set row
        elif not dxdm:
            #if dxdm flag is false it is the ncs1 dataset with is DS0001
            if x == 3: 
                start = ncs1.columns.get_loc(tree[x][0])
                # first iteration gives start value
            if x+1 == len(tree):
                end = ncs1.columns.get_loc(tree[x][0])
            #start and end values are the the location of the varnames at 3 and the end of the list
            # Setup temp DF
            tmpDF.loc[len(tmpDF)] = [tree[x][0], tree[x][1], var, ncs1.columns.get_loc(tree[x][0]), ncs1.columns.get_loc(tree[x][0]), 'ncs1', 0]
            # iterate through list and add rows to df
        else:  
            # otherwise the values are part of the ncsDXDM or DS0002 dataset
            if ncsDXDM.columns.get_loc(tree[x][0]) < start: 
                start = ncsDXDM.columns.get_loc(tree[x][0])
                # first iteration gives start value
            if ncsDXDM.columns.get_loc(tree[x][0]) > end:
                end = ncsDXDM.columns.get_loc(tree[x][0])
            #start and end values are the the location of the varnames at 3 and the end of the list
            for x in range(3, len(tree)):
                tmpDF.loc[len(tmpDF)] = [tree[x][0], tree[x][1], var, ncsDXDM.columns.get_loc(tree[x][0]), ncsDXDM.columns.get_loc(tree[x][0]), 'ncsDXDM', 0]
                # iterate through list and add rows to df
    setattr(root, var, tmpDF)
    return (start, end)


def search_for_description(var, dxdm, tree_data, lvl = 1):
    value = -99999999999999999
    description = ""
    if lvl: 
        if dxdm: 
            try: 
                value = ncsDXDM.columns.get_loc(var)
            except: 
                print("No value " + var + " in ncsDXDM (DS0002)")
                return
            tree_data = tree_data[tree_data.DataFrame == "ncsDXDM"]
        else: 
            try: 
                value = ncs1.columns.get_loc(var)
            except: 
                print("No value " + var + " in ncs1 (DS0001)")
                return
            tree_data = tree_data[tree_data.DataFrame == "ncs1"]
        for x in range(0, len(tree_data)):
            if value <=tree_data.iloc[x, 5] and value >= tree_data.iloc[x, 4]:
                if tree_data.iloc[x, 0] != var:
                    description = search_for_description(var, dxdm, eval("tree_root." + tree_data.iloc[x,0]),  0)
                elif tree_data.iloc[x, 0] == var: 
                    return tree_data.iloc[x,:]
                else: 
                    return -1
    else: 
        if dxdm: 
            try: 
                value = ncsDXDM.columns.get_loc(var)
            except: 
                print("No value " + var + " in ncsDXDM (DS0002)")
        else: 
            try: 
                value = ncs1.columns.get_loc(var)
            except: 
                print("No value " + var + " in ncs1 (DS0001)")
                return
        for x in range(0, len(tree_data)):
            if value <=tree_data.loc[x, 'End'] and value >= tree_data.loc[x, 'Start']:
                if tree_data.iloc[x, 0] != var and tree_data.loc[x, 'recursion_flag'] == 1:
                    description = search_for_description(var, dxdm, eval("tree_root." + tree_data.iloc[x,0]),  0)
                elif tree_data.iloc[x, 0] == var: 
                    return tree_data.iloc[x,:]
                else: 
                    return -1
    return description


In [271]:
search_for_description("AGONS", 1, tree_data)


VarName                                 AGONS
Description       AGO WITH OR W-O PANIC ONSET
Root_DF                      DXDM_AGORAPHOBIA
Start                                       6
End                                         6
DataFrame                             ncsDXDM
recursion_flag                              0
Name: 6, dtype: object

In [234]:
tree_data[:]

,VarName,Description,Value,DataFrame,Start,End
0,CASE_IDENTIFICATION_VARIABLES,CASE IDENTIFICATION VARIABLES,0,ncs1,0,0
1,DIAGNOSTIC_INTERVIEW_SURVEY_ADMINISTRATIONFACE...,DIAGNOSTIC INTERVIEW | SURVEY ADMINISTRATION/F...,1,ncs1,1,14
2,DIAGNOSTIC_INTERVIEW_SECTION_A_ACTIVITIES_OF_D...,DIAGNOSTIC INTERVIEW | SECTION A: ACTIVITIES O...,2,ncs1,15,78
3,DIAGNOSTIC_INTERVIEW_SECTION_B_LIFETIME_MOODS_...,DIAGNOSTIC INTERVIEW | SECTION B: LIFETIME MOO...,3,ncs1,79,363
4,DIAGNOSTIC_INTERVIEW_SECTION_C_ONGOING_SADNESS,DIAGNOSTIC INTERVIEW | SECTION C: ONGOING SADNESS,4,ncs1,364,377
5,DIAGNOSTIC_INTERVIEW_SECTION_D_SADNESS,DIAGNOSTIC INTERVIEW | SECTION D: SADNESS,5,ncs1,378,632
6,DIAGNOSTIC_INTERVIEW_SECTION_E_MANIA,DIAGNOSTIC INTERVIEW | SECTION E: MANIA,6,ncs1,633,739
7,DIAGNOSTIC_INTERVIEW_SECTION_F_ALCOHOL,DIAGNOSTIC INTERVIEW | SECTION F: ALCOHOL,7,ncs1,740,755
8,DIAGNOSTIC_INTERVIEW_SECTION_G_MEDICATIONS_amp...,DIAGNOSTIC INTERVIEW | SECTION G: MEDICATIONS ...,8,ncs1,756,1576
9,DIAGNOSTIC_INTERVIEW_SECTION_H_PROBLEM_BEHAVIORS,DIAGNOSTIC INTERVIEW | SECTION H: PROBLEM BEHA...,9,ncs1,1577,1593


In [91]:
tree_data = pd.DataFrame(columns = ['VarName', 'Description', 'Value', 'DataFrame', 'Start', 'End'])


In [92]:
#This builds an object representing variable names and all of their desriptions. format is as follows: 
  # 1. tree_data holds top level varnames & description & location in tree[] list
  # 2. root.tree_data.iloc[n, 0] holds the next level varnames, description, and either location in tree[] list or REF value if given
  # 3. root.(root.tree_data.iloc[n,0]).iloc[n,0] varies. For some is is the final level with a 1D list with the column index from ncs1, for others it is the varname for another level
  # If this level exists it generally represents the final information 
  # To traverse the object: 
    # eval('tree_root.' + varnamestring)
    # LVL1 EX: eval('tree_root.' + tree_data.iloc[4, 0])
    # LVL2 EX: eval('tree_root.' + eval('tree_root.' + tree_data.iloc[4, 0] + ".iloc[0,0]"))
tree_data_tmp = pd.DataFrame(columns = ['VarName', 'Description', 'Value', 'DataFrame'])
for i in range(0, len(tree)):
    varname = re.sub('[^ A-Za-z0-9]+', '', tree[i][0])
    varname = re.sub('[ ]+', '_', varname)
      # Get Variable name
    if i <= 30:
        # Set tree_data so that we can traverse object
      start_end = get_tree_obj_array(varname, tree_root, tree[i], 0)
      tree_data.loc[len(tree_data)] = [varname , tree[i][0], i, 'ncs1', start_end[0], start_end[1]]
    else:
        # Set tree_data so that we can traverse object
      start_end = get_tree_obj_array(varname, tree_root, tree[i], 1)
      tree_data.loc[len(tree_data)] = [varname , tree[i][0], i, 'ncsDXDM', start_end[0], start_end[1]]



In [235]:
eval('tree_root.' + tree_data.iloc[4, 0])

,VarName,Description,Root_DF,Start,End,DataFrame,recursion_flag
0,ONGOING_SADNESS_SCREENING_QUESTIONS,ONGOING SADNESS - SCREENING QUESTIONS,DIAGNOSTIC_INTERVIEW_SECTION_C_ONGOING_SADNESS,364,367,ncs1,1
1,ONGOING_SADNESS_AGE_OF_ONSET,ONGOING SADNESS - AGE OF ONSET,DIAGNOSTIC_INTERVIEW_SECTION_C_ONGOING_SADNESS,368,370,ncs1,1
2,ONGOING_SADNESS_SEVERITY_AND_DURATION,ONGOING SADNESS - SEVERITY AND DURATION,DIAGNOSTIC_INTERVIEW_SECTION_C_ONGOING_SADNESS,371,377,ncs1,1


In [9]:
ncsDXDM

,CASEID,SEASON,SWIN,SSPR,SSUM,SFAL,AGONS,AGONSA,AGOLT1,AGOLT2,...,RELNOPR,SEXF,SEX,SEXM,STR,SECU,P1FWT,P2WTV2,P2WTV3,OLDP2FWT
0,10001,4,0,0,0,1,,,0,0,...,0,1,2,0,34,2,0.5135,,,
1,10002,4,0,0,0,1,,,0,0,...,0,1,2,0,19,2,1.8968,,,
2,10003,4,0,0,0,1,,,0,0,...,0,1,2,0,39,2,0.4100,,,
3,10004,4,0,0,0,1,,,0,0,...,0,0,1,1,39,2,0.3093,,,
4,10005,4,0,0,0,1,,,0,0,...,0,0,1,1,7,1,0.7790,,,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8093,25952,4,0,0,0,1,,,0,0,...,0,1,2,0,30,1,3.1569,,,
8094,25953,3,0,0,1,0,,,0,0,...,0,1,2,0,2,2,0.1200,.0902,.2128,.0945
8095,25954,3,0,0,1,0,,,0,0,...,0,1,2,0,20,2,0.8161,.6145,.6435,.6439
8096,25955,4,0,0,0,1,,,0,0,...,0,0,1,1,1,2,0.5060,,,


In [10]:
ncs1

,CASEID,V2,V3,V4,V5,V6,V7,V8,V9,V10,...,V7437,V7438,V7439,V7440,V7441,V7442,P1FWT,P2WTV3,TOBACWT,P2TOBWT
0,10001,60080192,1,8128,9,28,90,9,28,90,...,9,9,9,9,99,9,0.5135,,0.5883,
1,10002,16030048,1,3941,10,1,90,10,1,90,...,9,9,9,9,99,9,1.8968,,2.1730,
2,10003,74070592,1,7813,10,5,90,10,5,90,...,9,9,9,9,99,9,0.4100,,0.4697,
3,10004,74030720,1,7813,10,10,90,10,10,90,...,9,9,9,9,99,9,0.3093,,0.3543,
4,10005,4180080,1,9018,10,10,90,10,10,90,...,9,9,9,9,99,9,0.7790,,0.8924,
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8093,25952,44110688,6,8331,11,8,91,11,8,91,...,0,0,0,0,0,0,3.1569,,3.6165,
8094,25953,1130821,5,4260,8,21,91,11,13,91,...,5,5,0,1,99,9,0.1200,.2128,0.1375,.3332
8095,25954,18070448,4,5319,7,7,91,7,7,91,...,0,0,0,0,0,0,0.8161,.6435,0.9349,1.0076
8096,25955,1400261,6,8224,11,22,91,11,22,91,...,5,5,0,2,0,0,0.5060,,0.5797,


In [26]:
tree_data.iloc[0,0]

'diagnostic_interview_survey_administrationfacesheet'

In [137]:
tree[0][2]
sampling_weight = pd.DataFrame(data = tree[0][2][3:len(tree[0][2])], columns = ['Index', 'Description'])

In [138]:
basic_info = pd.DataFrame(data = tree[1][3:len(tree[1])], columns = ['Index', 'Description', 'Other'])


In [201]:
a = pd.DataFrame(columns = ['Name', 'Range Start', 'Range End'])


In [202]:
social_life_health = pd.DataFrame(data = tree[2][3][3:len(tree[2][3])], columns = ['Index', 'Description', 'Other'])
a.loc[len(a)] = ['social_life_health', social_life_health.loc[0, 'Index'], social_life_health.loc[len(social_life_health)-1, 'Index'] ]


In [203]:
social_life_marriage = pd.DataFrame(data = tree[2][4][3:len(tree[2][4])], columns = ['Index', 'Description', 'Other'])
a.loc[len(a)] = ['social_life_marriage', social_life_marriage.loc[0, 'Index'], social_life_marriage.loc[len(social_life_marriage)-1, 'Index'] ]


In [204]:
social_life_relatives = pd.DataFrame(data = tree[2][5][3:len(tree[2][5])], columns = ['Index', 'Description', 'Other'])
a.loc[len(a)] = ['social_life_relatives', social_life_relatives.loc[0, 'Index'], social_life_relatives.loc[len(social_life_relatives)-1, 'Index'] ]


In [205]:
social_life_friends = pd.DataFrame(data = tree[2][6][3:len(tree[2][6])], columns = ['Index', 'Description', 'Other'])
a.loc[len(a)] = ['social_life_friends', social_life_friends.loc[0, 'Index'], social_life_friends.loc[len(social_life_friends)-1, 'Index'] ]


In [206]:
social_life_turnto = pd.DataFrame(data = tree[2][7][3:len(tree[2][7])], columns = ['Index', 'Description', 'Other'])
a.loc[len(a)] = ['social_life_turnto', social_life_turnto.loc[0, 'Index'], social_life_turnto.loc[len(social_life_turnto)-1, 'Index'] ]


In [207]:
checkpoint1 = pd.DataFrame(data = tree[2][8][3:len(tree[2][8])], columns = ['Index', 'Description', 'Other'])
a.loc[len(a)] = ['checkpoint1', checkpoint1.loc[0, 'Index'], checkpoint1.loc[len(checkpoint1)-1, 'Index'] ]


In [212]:
b = pd.DataFrame(columns = ['Name', 'Range Start', 'Range End'])

In [217]:
lifetime_frigt_anx = pd.DataFrame(data = tree[3][3][3:len(tree[3][3])], columns = ['Index', 'Description', 'Other'])
b.loc[len(b)] = ['lifetime_frigt_anx', lifetime_frigt_anx.loc[0, 'Index'], lifetime_frigt_anx.loc[len(lifetime_frigt_anx)-1, 'Index'] ]


In [224]:
lifetime_sad_blue_depress = pd.DataFrame(data = tree[3][4][3:len(tree[3][4])], columns = ['Index', 'Description', 'Other'])
b.loc[len(b)] = ['lifetime_sad_blue_depress', lifetime_sad_blue_depress.loc[0, 'Index'], lifetime_sad_blue_depress.loc[len(lifetime_sad_blue_depress)-1, 'Index'] ]

In [229]:
lifetime_lost_interest = pd.DataFrame(data = tree[3][5][3:len(tree[3][5])], columns = ['Index', 'Description', 'Other'])
b.loc[len(b)] = ['lifetime_lost_interest', lifetime_lost_interest.loc[0, 'Index'], lifetime_lost_interest.loc[len(lifetime_lost_interest)-1, 'Index'] ]

In [232]:
lifetime_manic_irrit = pd.DataFrame(data = tree[3][6][3:len(tree[3][6])], columns = ['Index', 'Description', 'Other'])
b.loc[len(b)] = ['lifetime_manic_irrit', lifetime_manic_irrit.loc[0, 'Index'], lifetime_manic_irrit.loc[len(lifetime_manic_irrit)-1, 'Index'] ]

In [238]:
too_strong_fears = pd.DataFrame(data = tree[3][7][3:len(tree[3][7])], columns = ['Index', 'Description', 'Other'])
b.loc[len(b)] = ['too_strong_fears', too_strong_fears.loc[0, 'Index'], too_strong_fears.loc[len(too_strong_fears)-1, 'Index'] ]

In [242]:
too_strong_fears_physical = pd.DataFrame(data = tree[3][8][3:len(tree[3][8])], columns = ['Index', 'Description', 'Other'])
b.loc[len(b)] = ['too_strong_fears_physical', too_strong_fears_physical.loc[0, 'Index'], too_strong_fears_physical.loc[len(too_strong_fears_physical)-1, 'Index'] ]

In [245]:
too_strong_fears_tell_dr = pd.DataFrame(data = tree[3][9][3:len(tree[3][9])], columns = ['Index', 'Description', 'Other'])
b.loc[len(b)] = ['too_strong_fears_tell_dr', too_strong_fears_tell_dr.loc[0, 'Index'], too_strong_fears_tell_dr.loc[len(too_strong_fears_tell_dr)-1, 'Index'] ]

In [248]:
too_strong_fears_take_meds = pd.DataFrame(data = tree[3][10][3:len(tree[3][10])], columns = ['Index', 'Description', 'Other'])
b.loc[len(b)] = ['too_strong_fears_take_meds', too_strong_fears_take_meds.loc[0, 'Index'], too_strong_fears_take_meds.loc[len(too_strong_fears_take_meds)-1, 'Index'] ]

In [ ]:
too_strong_fears_limit_life = pd.DataFrame(data = tree[3][7][3:len(tree[3][7])], columns = ['Index', 'Description', 'Other'])
b.loc[len(b)] = ['too_strong_fears', too_strong_fears.loc[0, 'Index'], too_strong_fears.loc[len(too_strong_fears)-1, 'Index'] ]

In [251]:
tree [2]

['DIAGNOSTIC INTERVIEW | SECTION A: ACTIVITIES OF DAILY LIFE',
 0,
 0,
 ['SOCIAL LIFE - HEALTH',
  0,
  0,
  ['V101', 'a1:rate physical health', 'REF= 101 ID=DDE'],
  ['V102', 'a2:rate mental health', 'REF= 102 ID=DDE'],
  ['V103', 'a3:rate health vs.others', 'REF= 103 ID=DDE'],
  ['V104', 'a5:freq balanced meals', 'REF= 104 ID=DDE'],
  ['V105', 'a6:#hrs sleep in 24 hrs', 'REF= 105 ID=DDE'],
  ['V106', 'a7:freq regular exercise', 'REF= 106 ID=DDE'],
  ['V107', 'a8:chkpt-spouse in HH', 'REF= 107 ID=DDE']],
 ['SOCIAL LIFE - MARRIAGE &amp; PARTNERSHIP',
  0,
  0,
  ['V108', "a9:R's marital status", 'REF= 108 ID=DDE'],
  ['V109', 'a9a:livng mrriag-likerel', 'REF= 109 ID=DDE'],
  ['V110', 'a9b:spouse lives here', 'REF= 110 ID=DDE'],
  ['V111', 'a9c:where does spouslive', 'REF= 111 ID=DDE'],
  ['V112', 'a9d:#time sps live there', 'REF= 112 ID=DDE'],
  ['V113', "a9d:per'd time sps there", 'REF= 113 ID=DDE'],
  ['V114', 'a9e:freq contact w/spous', 'REF= 114 ID=DDE'],
  ['V201', 'a10:spous care

In [246]:
b

,Name,Range Start,Range End
0,lifetime_frigt_anx,V301,V305
1,lifetime_sad_blue_depress,V306,V309
2,lifetime_lost_interest,V310,V311
3,lifetime_manic_irrit,V312,V313
4,too_strong_fears,V314,V320
5,too_strong_fears_physical,V321,V326
6,too_strong_fears_tell_dr,V327,V331
